In [1]:
import os, glob, math, imp
import numpy as np

In [2]:
# os.environ['TEMPO2'] = '/Users/vallis/miniconda3/envs/sksparse/share/tempo2'
os.environ['TEMPO2'] = '/Users/vallis/miniconda3/envs/nanograv/share/tempo2'
# os.environ['TEMPO2'] = '/usr/local/share/tempo2'

In [3]:
import enterprise
import enterprise.pulsar as ep
import enterprise.signals.parameter as esp
import enterprise.signals.signal_base as esb
import enterprise.signals.white_signals as esw

from enterprise.signals.signal_base import function, Function

In [4]:
# import pixiedust

## Tests in `test_hierarchical_parameter.py`

In [29]:
import tests.test_hierarchical_parameter

In [36]:
imp.reload(tests.test_hierarchical_parameter)

<module 'tests.test_hierarchical_parameter' from '/Users/vallis/Documents/enterprise/tests/test_hierarchical_parameter.py'>

In [37]:
cls = tests.test_hierarchical_parameter.TestHierarchicalParameter
cls.setUpClass()
ins = cls()

In [39]:
ins.test_enterprise_Parameter()

In [40]:
ins.test_enterprise_Function()

In [41]:
ins.test_Function_of_Function()

In [42]:
ins.test_powerlaw()

In [38]:
ins.test_powerlaw_equad()

## Enterprise `Parameter`

In [15]:
x = esp.Uniform(0,1)

In [21]:
issubclass(x,esp.Parameter)

True

In [11]:
x.get_logpdf(0.5)

TypeError: You can only call get_logpdf() on an instantiated (named) Parameter.

In [12]:
x1 = x('x1')

In [36]:
isinstance(x1,esp.Parameter)

True

In [40]:
str(x1) == 'x1:Uniform(pmin=0, pmax=1)'

True

In [41]:
x1.get_logpdf(0.5) == 0

True

## Enterprise `Function` (positional + keywork argument)

In [45]:
def add(a, x=1, y=2):
    return a + x + y

f = Function(add, x=x)

In [51]:
issubclass(f,esp.FunctionBase)

True

In [53]:
f1 = f('f1')

In [63]:
f1

f1(f1_x:Uniform(pmin=0, pmax=1))

In [61]:
isinstance(f1,esp.FunctionBase)

True

In [73]:
f1.params

[f1_x:Uniform(pmin=0, pmax=1)]

In [75]:
f1(2)

True

In [151]:
f1(2,0.5,7)

9.5

This is OK to fail, but seems the wrong error message.

In [92]:
f1.func_args

['a', 'x', 'y']

In [91]:
f1.func_kwargs

{'x': enterprise.signals.parameter.Uniform.<locals>.Uniform}

**Note this is unresolved, and possibly should remain so; the semantics of Function are rather complicated at this point. Maybe they should be simplified, even at the cost of complicating code elsewhere.**

In [95]:
f1(params={'a': 3})

TypeError: add() missing 1 required positional argument: 'a'

**This does not work either.**

In [121]:
f1(3,x=4)

9

In [118]:
f1(3,params={'f1_x': 4})

9

In [123]:
f1(3,0.5,7)

10.5

In [124]:
f1(3,x=0.5,y=7)

10.5

## `Function` with `Function` parameter 

In [131]:
def doub(x):
    return 2*x

f = Function(doub, x=esp.Uniform(0,1))

def mult(a, w=2, z=1):
    return a * w * z

g = Function(mult, w=esp.Uniform(2,3), z=f)

In [132]:
g1 = g('g1')

In [137]:
isinstance(g1,esp.FunctionBase)

True

In [138]:
g1

g1(g1_w:Uniform(pmin=2, pmax=3), g1_z_x:Uniform(pmin=0, pmax=1))

In [140]:
g1.params

[g1_w:Uniform(pmin=2, pmax=3), g1_z_x:Uniform(pmin=0, pmax=1)]

Does not use the z default... messy to fix because it would have to know that it does not have enough information to compute the function before calling it.

In [141]:
g1(2)

TypeError: doub() missing 1 required positional argument: 'x'

These work.

In [142]:
g1(2,z=3)

12

In [165]:
g1(2,w=10,z=3)

60

This also works. Should it? Potentially problematic if there are parameter name collisions...

In [143]:
g1(2,x=5)

40

In [144]:
g1(2,params={'g1_z_x': 5, 'g1_w': 10})

200

## Justin's use case

In [13]:
def powerlaw(f, log10_A=-15):
    return (10**log10_A) * f**2

pl = Function(powerlaw, log10_A=esp.Uniform(0,5))

In [153]:
pl1 = pl('pl1')

In [7]:
pl1

pl1(pl1_log10_A:Uniform(pmin=0, pmax=5))

In [155]:
fs = np.array([1,2,3])

In [156]:
pl1(fs)

array([1.e-15, 4.e-15, 9.e-15])

In [157]:
pl1(fs, log10_A=-16)

array([1.e-16, 4.e-16, 9.e-16])

Note the name of the parameter is not just log10_A. Is this the desired default behavior?

In [11]:
pl1(fs, params={'pl1_log10_A': -17})

array([  1.00000000e-17,   4.00000000e-17,   9.00000000e-17])

In [14]:
def log10(A=10**-16):
    return math.log10(A)

pm = Function(powerlaw, log10_A=Function(log10, A=esp.Uniform(10**-17,10**-14)))

In [165]:
pm1 = pm('pm1')

In [166]:
pm1

pm1(pm1_log10_A_A:Uniform(pmin=1e-17, pmax=1e-14))

In [167]:
pm1(fs, log10_A=-13)

array([1.e-13, 4.e-13, 9.e-13])

Note this uses the log10 default, not the powerlaw default... that's OK I think since the log10_A = Function specification comes later.

In [16]:
pm1(fs)

array([  1.00000000e-16,   4.00000000e-16,   9.00000000e-16])

Finally this is what Justin cares about. The naming of the parameter is somewhat abstruse... 

In [168]:
pm1(fs, params={'pm1_log10_A_A': 10**-19})

array([1.e-19, 4.e-19, 9.e-19])

But it doesn't work (yet) with `Signal` parameters.

In [5]:
datadir = 'tests/data'

psr = ep.Pulsar(datadir + '/B1855+09_NANOGrav_9yv1.gls.par',
                datadir + '/B1855+09_NANOGrav_9yv1.tim')

In [6]:
equad = esw.EquadNoise(log10_equad=esp.Normal(0,10))

In [7]:
equad1 = equad(psr)

In [8]:
equad1.params

[B1855+09_log10_equad:Normal(mu=0, sigma=10)]

In [10]:
equad1.get_ndiag(params={'B1855+09_log10_equad': -12})[:10]

ndarray_alt([1.e-24, 1.e-24, 1.e-24, 1.e-24, 1.e-24, 1.e-24, 1.e-24,
             1.e-24, 1.e-24, 1.e-24])

Let's try with a `Function` parameter

In [15]:
fquad = esw.EquadNoise(log10_equad=Function(log10, A=esp.Uniform(10**-17,10**-14)))

In [16]:
fquad1 = fquad(psr)

In [18]:
fquad1.params

[B1855+09_log10_equad_A:Uniform(pmin=1e-17, pmax=1e-14)]

In [23]:
fquad1.get_ndiag(params={'B1855+09_log10_equad_A': 10**-14})[:4]

ndarray_alt([1.e-28, 1.e-28, 1.e-28, 1.e-28])

## Demo for PR

In [46]:
import enterprise.signals.parameter as esp
import enterprise.signals.signal_base as ebb

In [45]:
@esb.function
def gaussianprior(x, x0=0, std=1):
    return np.exp(-0.5 * (x - x0)**2 / std**2) / np.sqrt(2 * math.pi * std**2)

In [47]:
b = esp.UserParameter(gaussianprior(x0=esp.Uniform(-1,1), std=esp.Uniform(-2,2)))

In [48]:
b1 = b('gauss')

In [49]:
b1

gauss:UserParameter

In [50]:
b1.params

[gauss:UserParameter,
 gauss_x0:Uniform(pmin=-1, pmax=1),
 gauss_std:Uniform(pmin=-2, pmax=2)]

In [51]:
b1.get_pdf(0.5)

0.3520653267642995

In [52]:
b1.get_pdf(params={'gauss': 0.5, 'gauss_x0': 1, 'gauss_std': 2})

0.19333405840142465

In [53]:
def f(x, a=0):
    return x + a

In [54]:
c = esb.Function(f,a = esp.UserParameter(gaussianprior(x0=esp.Uniform(-1,1), std=esp.Uniform(-2,2))))

In [55]:
c1 = c('secondary')

In [56]:
c1

secondary(secondary_a:UserParameter, secondary_a_x0:Uniform(pmin=-1, pmax=1), secondary_a_std:Uniform(pmin=-2, pmax=2))

In [57]:
c1.params

[secondary_a:UserParameter,
 secondary_a_x0:Uniform(pmin=-1, pmax=1),
 secondary_a_std:Uniform(pmin=-2, pmax=2)]

In [60]:
c1(0,params={'secondary_a': 2,'secondary_a_x0': -1})

2

## Earlier tests, disregard...

In [5]:
@function
def gaussianprior(x, x0=0, std=1):
    return np.exp(-0.5 * (x - x0)**2 / std**2) / np.sqrt(2 * math.pi * std**2)

In [24]:
@esb.function
def pl(f,log10_A=2):
    return f**(2*log10_A)

In [30]:
@esb.function
def log10(A):
    return np.log10(A)

In [51]:
F = pl(log10_A=log10(A=esp.Uniform(0,1)))

In [54]:
iF = F('F')

In [55]:
iF.params

[]

In [7]:
a = esp.Uniform(0,1)

In [8]:
a1 = a('uniform')

In [9]:
a1

"uniform":Uniform(0,1)

In [10]:
a1.params

["uniform":Uniform(0,1)]

In [15]:
a1.get_pdf(0.5)

1.0

In [11]:
b = esp.UserParameter(gaussianprior(x0=esp.Uniform(-1,1), std=esp.Uniform(-2,2)))

In [12]:
b1 = b('gauss')

In [13]:
b1

"gauss":UserParameter(gauss_std,gauss_x0)

In [14]:
b1.params

["gauss":UserParameter(gauss_std,gauss_x0),
 "gauss_std":Uniform(-2,2),
 "gauss_x0":Uniform(-1,1)]

In [16]:
b1.get_pdf(0.5)

0.35206532676429952

In [17]:
b1.get_pdf(params={'gauss': 0.5, 'gauss_x0': 0, 'gauss_std': 1})

0.35206532676429952

In [18]:
b1.get_pdf(params={'gauss': 0.5, 'gauss_x0': 1, 'gauss_std': 2})

0.19333405840142465

In [19]:
def f(x, a=0):
    return x + a

In [21]:
c = esb.Function(f,a = esp.UserParameter(gaussianprior(x0=esp.Uniform(-1,1), std=esp.Uniform(-2,2))))

In [22]:
c1 = c('secondary')

In [23]:
c1.params

["secondary_a":UserParameter(secondary_a_std,secondary_a_x0),
 "secondary_a_std":Uniform(-2,2),
 "secondary_a_x0":Uniform(-1,1)]

In [30]:
c1(0,params={'secondary_a': 2})

2